In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ml-finalexam-p5/train.csv
/kaggle/input/2022-ml-finalexam-p5/test.csv
/kaggle/input/2022-ml-finalexam-p5/sample_submit.csv


In [2]:
train = pd.read_csv('/kaggle/input/2022-ml-finalexam-p5/train.csv')
test = pd.read_csv('/kaggle/input/2022-ml-finalexam-p5/test.csv')
sample = pd.read_csv('/kaggle/input/2022-ml-finalexam-p5/sample_submit.csv')

train데이터와 test데이터, 제출할 파일을 읽어 오기 위해서 pd.read_csv함수를 사용하였습니다.

In [3]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
595,596,0,3,"Van Impe, Mr. Jean Baptiste",male,36.0,1,1,345773,24.1500,NaN,S
596,597,1,2,"Leitch, Miss. Jessie Wills",female,NaN,0,0,248727,33.0000,NaN,S
597,598,0,3,"Johnson, Mr. Alfred",male,49.0,0,0,LINE,0.0000,NaN,S
598,599,0,3,"Boulos, Mr. Hanna",male,NaN,0,0,2664,7.2250,NaN,C


In [4]:
X_train = train.drop(['Survived','Cabin','Name'],axis=1)
X_test = test.drop(['Cabin','Name'],axis=1)
y_train = train['Survived']

train데이터와 test데이터를 살펴보니 'Survived','Cabin','Name'라는 부분은 모델의 학습에 필요하지 않을 것 같아서 제거해 주기 위해서 drop 함수를 사용하였습니다.

In [5]:
X_train.info()
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  600 non-null    int64  
 1   Pclass       600 non-null    int64  
 2   Sex          600 non-null    object 
 3   Age          474 non-null    float64
 4   SibSp        600 non-null    int64  
 5   Parch        600 non-null    int64  
 6   Ticket       600 non-null    object 
 7   Fare         600 non-null    float64
 8   Embarked     599 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 42.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291 entries, 0 to 290
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  291 non-null    int64  
 1   Pclass       291 non-null    int64  
 2   Sex          291 non-null    object 
 3   Age          240 non-null    float64
 4   SibSp     

train데이터와 test데이터의 각각의 데이터형식과 null값의 개수를 파악하기 위해서 info함수를 사용하였습니다.

In [6]:
from sklearn import preprocessing
label = ['Sex','Ticket','Embarked']
box = pd.concat([X_train,X_test])
for i in label:
    le = preprocessing.LabelEncoder()
    le.fit(box[i])
    X_train[i] = le.transform(X_train[i])
    X_test[i] = le.transform(X_test[i])

데이터에 있는 값들 중 컴퓨터가 이해할 수 없는 한글을 실수화 해주기 위해서 labelencoder 함수를 사용했습니다.

In [7]:
bow = X_train[X_train['Age'].isnull()].index
ages = X_train['Age'].copy()
for i in bow:
    ages[i] = -1
X_train['Age'] = ages

train데이터의 'Age'라는 부분에 'NaN'값이 들어가 있어서 해당 값들을 찾아내고 -1로 교환하고자 index함수와 copy함수를 사용했습니다.

In [8]:
bow = X_test[X_test['Age'].isnull()].index
tages = X_test['Age'].copy()
for i in bow:
    tages[i] = -1
X_test['Age'] = tages

test데이터의 'Age'라는 부분에 'NaN'값이 들어가 있어서 해당 값들을 찾아내고 -1로 교환하고자 index함수와 copy함수를 사용했습니다.

In [9]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(random_state=42).fit(X_train,y_train)
y_test = model.predict(X_test)

여러가지의 분류기중 최근 가장 좋은 성능을 보이고 있다는 lightgbm을 사용하기 위해서 해당 모델을 호출하여 학습하고 예측을 진행했습니다.

In [10]:
sample['Survived'] = y_test
sample.to_csv('submit.csv',index=False)

예측한 값을 csv파일로 옮겨서 제출하기 위해서 to_csv함수를 사용했습니다.